In [3]:
#Pacotes necessários para a execução do código

import pdal
import glob
import geopandas as gpd
import numpy as np
import json
import pandas as pd
from shapely import MultiPoint
import alphashape
import shapely

#Resolução para arquivos raster
resolution = 0.5

#Carregar dados de LiDAR .laz a partir de uma pasta 'downloads/LiDAR/*.laz'
files = glob.glob('downloads/LiDAR/*.laz')

#Unir dados de LiDAR num arquivo só
!pdal merge {(' ').join(files)} downloads/LiDAR/san_remo.laz

**Função dem_pipeline(resolution)**
- **Objetivo**:
    - Recebe dados de LiDAR e gera um Modelo Digital de Elevação (DEM)
    - Recebe como valor de entrada a resolução definida anteriormente


**1. Leitura dos dados LiDAR**
- "type": "readers.las" → indica que o pipeline vai ler dados LAS (arquivo que armazena dados LiDAR) ou LAZ (versão comprimida)
- "filename": f'downloads/LiDAR/san_remo.laz' → especifica o caminho para o arquivo LiDAR comprimido (.laz)
- "override-srs": "EPSG:31983" → define o sistema de referência espacial para os dados, nesse caso SIRGAS 2000/UTM zone 23S

**2. Filtragem por Classificação**
- "type": "filters.range" → aplica um filtro baseado em intervalos de valores
- "limits": "Classification[2:2]" → filtra os pontos classificados com valor 2, que correspondem aos dados de solo, do terreno, em classificações LiDAR padrão

**3. Triangulação de Delaunay**
- "type": "filters.delaunay" → realiza triangulação de Delaunay sobre os pontos filtrados. Isso cria uma malha triangular que cobre os pontos, essencial para a interpolação da superfície do terreno

**4. Rasterização da Face**
- "type": "filters.faceraster" → converte a malha triangular resultante em um raster que representa a superfície do terreno
- "resolution": resolution → define o tamanho da célula do raster a partir do parâmetro de entrada da função

**5. Escrita do Raster DEM**
- "filename": f"results/DEM/MDT-san_remo-50cm.tiff" → define o caminho e nome do arquivo de saída
    - ATENÇÃO: essas pastas "results" e "DEM" PRECISAM ser criadas na pasta do teste do código antes de rodá-lo
- "gdaldrive": "GTiFF" → especifica que o formato do arquivo será GeoTIFF, um formato raster georreferenciado
- "type": "writers.raster" → escreve o raster gerado para um arquivo
- "gdalopts":"COMPRESS=ZSTD, PREDICTOR=3, BIGTIFF=YES" → define opções do pacote GDAL
    - compress=zstd: usa compressão Zstandard para reduzir o tamanho do arquivo
    - predictor=3: otimiza a compressão para dados de ponto flutuante
    - bigtiff=yes: permite a criação de arquivos GeoTIFF maiores que 4 GB
- "nodata": "0" → define como valor nulo o que representa ausência de dados no raster
- "data_type": "float32" → float32 indica que os valores do DEM serão armazenados como números de ponto flutuante de 32 bits

In [7]:
def dem_pipeline(resolution):
    return [
        {
            "type": "readers.las",
            "filename": f'downloads/LiDAR/san_remo.laz',
            "override_srs": "EPSG:31983"
        },
        {
            "type": "filters.range",
            "limits": "Classification[2:2]"
        },
        {
            "type": "filters.delaunay"
        },
        {
            "type": "filters.faceraster",
            "resolution": resolution
        },
        {
            "filename": f"results/DEM/MDT-san_remo-50cm.tiff",
            "gdaldriver": "GTiff",
            "type": "writers.raster",
            "gdalopts": "COMPRESS=ZSTD, PREDICTOR=3 BIGTIFF=YES",
            "nodata": "0",
            "data_type": "float32"
        }
    ]

**Função laz_pipeline(resolution)**
- Gera diferentes tipos de arquivos raster e promove clusterização nos pontos do arquivo LiDAR

**1. Leitura dos dados LiDAR**
- "type": "readers.las" → indica que o pipeline vai ler dados LAS (arquivo que armazena dados LiDAR) ou LAZ (versão comprimida)
- "filename": f'downloads/LiDAR/san_remo.laz' → especifica o caminho para o arquivo LiDAR comprimido (.laz)

**2. Escrita de um Raster BHM-Z**
- **Objetivo**: gerar um raster representando a altura máxima dos pontos classificados como 6 (edificações) na dimensão Z
- "filename": f"results/BHM-Z-san_remo.tiff" → define o formato .tiff como arquivo de saída
- "gdaldriver":"GTiff" → especifica o formato GeoTIFF
- "radius": f'{resolution * 2 * np.sqrt(2)}' → define o raio para agregação dos pontos. É calculado como resolution * 2 * raiz(2)
- "override_srs": "EPSG:31983" → define sistema de referência espacial para SIRGAS 2000 
- "output_type":"max" → o valor máximo dentro do raio definido será usado para cada célula do raster
- "resolution":resolution → define a resolução espacial do raster
- "dimension": "Z" → indica que a dimensão de elevação (Z) será utilizada
- "data_type": "float32" → especifica que os valores serão armazenados como números de ponto flutuante de 32 bits
- "type": "writers.gdal" → utiliza GDAL para escrever dados raster
- "gdalopts":"COMPRESS=ZSTD, PREDICTOR=3, BIGTIFF=YES" → define opções do pacote GDAL
    - compress=zstd: usa compressão Zstandard para reduzir o tamanho do arquivo
    - predictor=3: otimiza a compressão para dados de ponto flutuante
    - bigtiff=yes: permite a criação de arquivos GeoTIFF maiores que 4 Gb
- "where": "(Classification == 6)" → filtra os pontos com classificação igual a 6, que representa edificações

**3. Cálculo da Altura Acima do Solo (HAG)**
- "type":"filters.hag_dem" → calcula a High Above Ground (HAG), ou seja, a altura dos objetos acima do terreno
- "raster": f"results/DEM/MDT-san_remo-50cm.tiff" → utiliza do DEM (Modelo Digital de Elevação) gerado anteriormente para referenciar o solo

**4. Escrita de um Raster BHM**
- **Objetivo**: gerar um raster representando a altura das edificações em relação ao solo
- "filename":f"results/BHM-san_remo.tiff" → formato .tiff do arquivo de saída
- "gdaldriver":"GTiff"
- "output_type":"max" → o valor máximo dentro do raio definido será usado para cada célula do raster
- "resolution": resolution
- "radius": f'{resolution * 2 * np.sqrt(2)}' → define raio para agregação dos pontos
- "dimension":"HeightAboveGround" → a altura atribuída consiste na altura calculada no passo anterior, ou seja, a altura das edificações com relação ao solo
- "data_type": "float32"
- "type": "writers.gdal"
- "where": "(Classification == 6)" → pontos classificados como edificações
- "override_srs": "EPSG:31983"

**5. Filtragem por Classificação**
- **Objetivo**: manter apenas os pontos 6, de edificações
- "type":"filters.range"
- "limits":"Classification[6:6]"

**6. Redução de densidade de pontos com Voxel Downsize**
- **Objetivo**: diminuir a quantidade de pontos para facilitar o processamento subsequente
- "type":"filters.voxeldownsize" → reduz a densidade da nuvem de pontos usando uma grade voxel. Voxel é um ponto tridimensional em uma grade cartesiana. O termo vem da junção das palavras "volume" e "pixel"
- "cell":0.5 → define o tamanho da célula (confirmar se são em metros)
- "mode":"center" → seleciona o ponto central dentro de cada voxel para representar o voxel reduzido

**7. Clustering com DBSCAN**
- **Objetivo**: identificar cluster de pontos usando o algortimo DBSCAN
- "type":"filters.dbscan" → aplica o algoritmo DBSCAN para identificar cluster na nuvem de pontos
- "min_points":5 → número mínimo de pontos para formar um cluster
- "eps": (resolution + 0.10) * np.sqrt(2) → distância máxima entre dois pontos para que sejam considerados no mesmo cluster
- "dimensions":"X,Y,Z" → utiliza as três dimensões espaciais x, y e z para o clustering

**8. Transporte de dimensões com Ferry**
- **Objetivo**: ajustar os valores de altura após o clustering
- "type":"filters.ferry" → transfere os valores de uma dimensão para outra
- "dimensions":"HeightAboveGround => Z" → substitui os valores da coordenada Z (altura com relação ao nível do mar) pela altura com relação ao solo

**9. Escrita dos pontos clusterizados**
- **Objetivo**: salva a nuvem de pontos processada com informações clusterizadas
- "type":"writers.las" → escreve a nuvem de pontos com clusterização de volta para um arquivo .las
- "filename":f"results/Cluster-san_remo.laz" → formato do arquivo de saída
- "extra_dims": "all" → o "all" inclui todas as dimensões adicionais geradas durante o processamento, como ClusterID do DBSCAN

In [9]:
def laz_pipeline(resolution):
    return [
        {
            "type":"readers.las",
            "filename":f"downloads/LiDAR/san_remo.laz"
        },
        {
            "filename":f"results/BHM-Z-san_remo.tiff",
            "gdaldriver":"GTiff",
            "radius": f'{resolution * 2 * np.sqrt(2)}',
            "override_srs": "EPSG:31983",
            "output_type":"max",
            "resolution":resolution,
            "dimension": "Z",
            "data_type": "float32",
            "type": "writers.gdal",
            "gdalopts":"COMPRESS=ZSTD, PREDICTOR=3, BIGTIFF=YES",
            "where": "(Classification == 6)",
        },
        {
            "type":"filters.hag_dem",
            "raster": f"results/DEM/MDT-san_remo-50cm.tiff"
        },
        {
            "filename":f"results/BHM-san_remo.tiff",
            "gdaldriver":"GTiff",
            "output_type":"max",
            "resolution": resolution,
            "radius": f'{resolution * 2 * np.sqrt(2)}',
            "dimension":"HeightAboveGround",
            "data_type": "float32",
            "type": "writers.gdal",
            "where": "(Classification == 6)",
            "override_srs": "EPSG:31983"
        },
        {
            "type":"filters.range",
            "limits":"Classification[6:6]"
        },
        {
            "type":"filters.voxeldownsize",
            "cell":0.5,
            "mode":"center"
        },
        {
            "type":"filters.dbscan",
            "min_points":9,
            "eps": ((resolution * np.sqrt(2)) + 0.1),
            "dimensions":"X,Y,Z"
        },
        {
            "type":"filters.ferry",
            "dimensions":"HeightAboveGround => Z"
        },
        {
            "type":"writers.las",
            "filename":f"results/Cluster-san_remo.laz",
            "extra_dims": "all",
        },
    ]

**Estrutura geral do código**
1. Definição de Dicionários de Agregação e Renomeação de Colunas
2. Processamento com Pipelines PDAL (DEM e LAZ)
3. Manipulação de Dados com Pandas
4. Agregação e Conversão para GeoDataFrame
5. Exportação dos Resultados

**1. Definição de Dicionários de Agregação e Renomeação de Colunas**
- **Objetivo**: definir como os dados serão agregados após o agrupamento em cluster e como as colunas resultantes serão renomeadas para melhor legibilidade
- agg → é um dicionário que define como os dados serão agregados após o agrupamento. As chaves são os nomes das colunas e os valores especificam as operações de agregação a serem aplicadas
    - 'coords': list → agrupa as coordenadas em listas
    - 'Z': ['count', 'median', 'sum'] → conta o número de pontos (count), calcula a mediana (median) e soma (sum) os valores de Z
    - 'Intensity': 'median' → calcula a mediana do atributo
    - 'Red': 'median' 
    - 'Green': 'median' 
    - 'Blue': 'median' 

- columns → é um dicionário que mapeia os nomes das colunas resultantes após a agregação para nomes mais legíveis
    - ('coords', 'list'): 'coords' → renomeia ('coords', 'list') para somente 'coords'
    - ('Z', 'count'): 'count' 
    - ('Z', 'median'): 'z_median' 
    - ('Z', 'sum'): 'z_sum' 
    - ('Intensity', 'median'): 'intensity_median' 
    - ('Red', 'median'): 'red_median' 
    - ('Green', 'median'): 'green_median'  
    - ('Blue', 'median'): 'blue_median' 


**2. Processamento com Pipelines PDAL (DEM e LAZ)**
- **Objetivo**: executar as funções detalhadas nos dois códigos anteriores
- print('processando') → exibe uma mensagem indicando que o processamento dos dados está iniciando
- dem = dem_pipeline(resolution) → chama a função *dem_pipeline*
    - pipeline = pdal.Pipeline(json.dumps(dem)) → cria um objeto Pipeline da biblioteca PDAL, passando o Pipeline para o formato JSON
    - n_points = pipeline.execute() → executa o pipeline PDAL, processando os dados conforme definido na função *dem_pipeline*
    - print(f'Pipeline selected {n_points} points') → exibe o número de pontos processados pelo pipeline
- laz = laz_pipeline(resolution) → chama a função *laz_pipeline*
    - pipeline = pdal.Pipeline(json.dumps(laz)) → cria um objeto Pipeline da biblioteca PDAL, passando o Pipeline para o formato JSON
    - n_points = pipeline.execute() → executa o pipeline PDAL
    - print(f'Pipeline selected {n_points} points') → exibe o número de pontos processados pelo pipeline LAZ


**3. Manipulação de Dados com Pandas**
- **Objetivo**: converter os dados processados em um DataFrame, adicionar coordenadas, remover duplicatas e filtrar pontos com valores de Z específicos
- arr = pipeline.arrays[0] → obter uma array de pontos resultante da execução do pipeline PDAL
- df = pd.DataFrame(arr) → converter a array de pontos em um DataFrame do Pandas para facilitar a manipulação dos dados
- df.loc[:, 'coords'] = list(np.dstack([df.X, df.Y])[0])
    - np.dstack([df.X, df.Y]) → empilhar as colunas X e Y ao longo da terceira dimensão, criando pares de coordenadas
    - list(...[0]) → converte o resultado em uma lista de coordenadas
    - df.loc[:, 'coords'] = ... → adicionar uma nova coluna 'coords' ao DataFrame contendo as coordenadas (X, Y) para cada ponto
- df.set_index(['X', 'Y']).loc[:, 'Z'] = df.groupby(['X', 'Y']).agg({'Z': 'max'})
    - df.set_index(['X', 'Y']) → definir as colunas X e Y como índices do DataFrame
    - df.groupby(['X', 'Y']).agg({'Z': 'max'}) → agrupar os dados por X e Y e calcular o valor máximo de Z para cada grupo
    - df.loc[:, 'Z'] = ... → atualizar a coluna Z no DataFrame com os valores máximos calculados
- df.drop_duplicates(subset=['X', 'Y'], keep='last', inplace=True) → remover duplicatas de pontos com as mesmas coordenadas X e Y, mantendo apenas o último ponto
- df = df[(df.Z > 2.0) & (df.Z < 200.0)].reset_index()
    - df[(df.Z > 2.0) & (df.Z < 200.0)] → filtrar os pontos para manter apenas aqueles com Z entre 2.0 e 200.0
    - .reset_index() → resetar os índices do DataFrame após o filtro
  
**4. Agregação e Conversão para GeoDataFrame**
- **Objetivo**: agrupar os dados por ClusterID, aplicar as agregações definidas, renomear colunas, criar geometrias MultiPoint e converter para um GeoDataFrame
- df_agg = df[df.ClusterID > 0].groupby('ClusterID').agg(agg)
    - df[df.ClusterID > 0] → filtrar os pontos para manter apenas aqueles que pertencem a um cluster (assumindo que ClusterID > 0 indica pertencimento a um cluster)
    - .groupby('ClusterID').agg(agg) → agrupar os dados pelo ClusterID e aplicar as operações de agregação definidas no dicionário agg
- df_agg.columns = df_agg.columns.to_flat_index() → achatar os índices de múltiplos níveis resultantes da agregação para facilitar o acesso às colunas
- df_agg.rename(columns=columns, inplace=True) → renomear as colunas do DataFrame de acordo com o mapeamento definido no dicionário columns
- df_agg.loc[:, 'geometry'] = df_agg.coords.apply(MultiPoint)
    - df_agg.coords.apply(MultiPoint) → converter a lista de coordenadas em objetos MultiPoint do GeoPandas
    - df_agg.loc[:, 'geometry'] = ... → adicionar uma nova coluna 'geometry' contendo os objetos MultiPoint
- gdf_agg = gpd.GeoDataFrame(df_agg) → converter o DataFrame Pandas em um GeoDataFrame do GeoPandas, que suporta operações geoespaciais
- gdf_agg.set_crs(epsg=31983, inplace=True) → definir o Sistema de Referência de Coordenadas (CRS) do GeoDataFrame para EPSG:31983, que é o SIRGAS 2000 / UTM zone 23S

**5.Exportação dos Resultados**
- **Objetivo**: exportar o GeoDataFrame final para um arquivo GeoPackage, o que facilita o uso em softwares como o QGis e outras análises geoespaciais
- gdf_agg.drop(columns=['coords']).to_file(f'results/san_remo-multipoint.gpkg', driver='GPKG')
    - gdf_agg.drop(columns=['coords']) → remover a coluna 'coords' do GeoDataFrame, pois as informações de geometria já estão contidas na coluna 'geometry'. 
    - to_file → método do GeoPandas para exportar o GeoDataFrame para um arquivo
    - f'results/san_remo-multipoint.gpkg' → caminho e nome do arquivo de saída. gpkg indica que o formato será GeoPackage.
    - driver='GPKG' → especificar que o formato de saída é GeoPackage

In [11]:
agg = {
    'coords':list,  
    'Z':['count', 'median', 'sum'], 
    'Intensity':'median', 
    'Red':'median',
    'Green':'median',
    'Blue':'median'  
}

columns = {
    ('coords', 'list'):'coords',
    ('Z', 'count'):'count',
    ('Z', 'median'):'z_median',
    ('Z', 'sum'):'z_sum',
    ('Intensity', 'median'):'intensity_median',
    ('Red', 'median'):'red_median',
    ('Green', 'median'):'green_median',
    ('Blue', 'median'):'blue_median',
}

print('processando')

dem = dem_pipeline(resolution)
pipeline = pdal.Pipeline(json.dumps(dem))

laz = laz_pipeline(resolution)
pipeline = pdal.Pipeline(json.dumps(laz))


arr = pipeline.arrays[0]
df = pd.DataFrame(arr)
df.loc[:, 'coords'] = list(np.dstack([df.X, df.Y])[0])
df.set_index(['X', 'Y']).loc[:, 'Z'] = df.groupby(['X', 'Y']).agg({'Z':'max'})
df.drop_duplicates(subset=['X', 'Y'], keep='last', inplace=True)
df = df[(df.Z > 2.0) & (df.Z < 200.0)].reset_index()

df_agg = df[df.ClusterID > 0].groupby('ClusterID').agg(agg)
df_agg.columns = df_agg.columns.to_flat_index()
df_agg.rename(columns=columns, inplace=True)
df_agg.loc[:, 'geometry'] = df_agg.coords.apply(MultiPoint)
gdf_agg = gpd.GeoDataFrame(df_agg)
gdf_agg.set_crs(epsg=31983, inplace=True)
gdf_agg.drop(columns=['coords']).to_file(f'results/san_remo-multipoint.gpkg', driver='GPKG')



processando
Pipeline selected 6349330 points
Pipeline selected 1866807 points


C:\Users\User\AppData\Local\Temp\ipykernel_39996\1348440937.py:38: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  df.set_index(['X', 'Y']).loc[:, 'Z'] = df.groupby(['X', 'Y']).agg({'Z':'max'})
C:\Users\User\AppData\Local\Temp\ipykernel_39996\

**Processamentos finais**

**Estrutura geral do código**
1. Remoção da coluna 'coords'
2. Criação de Concave Hulls (envoltórias côncavas) para cluster significativos
3. Armazenamento das geometrias originais
4. Substituição das geometrias pelo Concave Hull
5. Exportação das geometrias poligonais
6. Restauração das geometrias originais

**1. Remoção da coluna 'coords'**
- **Objetivo**: remover a coluna 'coords' do DataFrame
- gdf_agg.drop(columns=['coords']) → a função 'drops' remove a coluna 'coords' do GeoDataFrame gdf_agg


**2. Criação de Concave Hulls (envoltórias côncavas) para cluster significativos**
- **Objetivo**: filtrar cluster com 16 ou mais pontos e aplicar uma função para criar concave hulls para esses clusters
- ashapes = gdf_agg.loc[df_agg.loc[:, 'count'] >= 16].coords.apply( lambda x: shapely.concave_hull(MultiPoint(x), ratio=0.1, allow_holes=Tru))
    - gdf_agg.loc[df_agg.loc[:, 'count'] >= 16] → selecionar os clusters (gdf_agg) nos quais a contagem de pontos (count) é maior ou igual a 16
    - coords → coluna que contém listas de coordenas [X, Y] para cada cluster
    - apply(lambda x: ...) → aplicar uma função lambda a cada lista de coordenadas. Uma função lambda é uma maneira de escrever uma função em uma única expressão, sem a necessidade de defini-la explicitamente
    - Multipoint(x) → criar um objeto MultiPoint do Shapely a partir das coordenadas
    - shapely.concave_hull(...) → criar a envoltória côncava dos pontos
)



**3. Armazenamento das geometrias originais**
- **Objetivo**: salvar temporariamente as geometrias originais para restaurá-las após a expor- geometry = gdf_agg.geometry → salvar temporariamente as geometrias originais do GeoDataFrame gdf_agg para restaurá-las posteriormente
  tação



**4. Substituição das geometrias pelo Concave Hull**
- **Objetivo**: atualizar as geometrias do GeoDataFrame para as concave hulls criadas, apenas para os clusters fil- gdf_agg.geometry = ashapes → substituir a coluna 'geometry' do GeoDataFrame gdf_agg pelas geometrias de concave hull geradas para os cluster filtrados. Para os cluster com 16 ou mais pontos, as geometrias são representadas por concave hulls, enquanto os demais permanecem inalterados
rados



**5. Exportação das geometrias poligonais**
- **Objetivo**: exportar o GeoDataFrame modificado para um arquivo GeoPackage contendo os polígonos das áreas ag- gdf_agg.drop(columns=['coords']).to_file(f'results/san_remo-multipoligono.gpkg', driver='GPKG')    - drop(columns=['coords']) → remover a coluna 'coords' do GeoDataFrame
    - .to_file(..., driver='GPKG') → exportar o GeoDataFrame para um arquivo GeoPackage (.gpkg), um formato de contêiner para dados geoespaciais, que suporta múltiplas camadas e formatos de dados
rupadas



**6. Restauração das geometrias originais**
- **Objetivo**: reverter as geometrias para as suas formas originais (MultiPoint), permitindo que o GeoDataFrame continue a ser usado com suas geometrias iniciais para operações- gdf_agg.geometry = geometry → restaurar a coluna 'geometry' original do GeoDataFrame gdf_agg, revertendo as geometrias de concave hull para as geometrias iniciais (MultiPoint). Isso permite que o GeoDataFrame gdf_agg continue sendo utilizado com suas geometrias originais para operações futuras, sem a necessidade de manter as alterações temporárias feitas para exportação.
 futuras


In [13]:
gdf_agg.drop(columns=['coords'])

ashapes = gdf_agg.loc[df_agg.loc[:, 'count'] >= 16].coords.apply(lambda x: shapely.concave_hull(MultiPoint(x), ratio=0.1, allow_holes=True))

gdf_agg.geometry = ashapes

gdf_agg.drop(columns=['coords']).to_file(f'results/san_remo-multipoligono.gpkg', driver='GPKG')

gdf_agg.geometry = geometry